In [0]:
# Create widgets for user input to parameterize notebook execution (recommended for reusability and automation)
dbutils.widgets.text("meta_file_path", "", "Path to the metadata JSON file")
dbutils.widgets.text("utilities_folder_path", "", "Utilities folder path")
dbutils.widgets.text("log_level", "INFO", "Log level")
dbutils.widgets.text("task_sys_params_out", "", "Task System Parameters")

# Retrieve widget values for use in downstream code (avoid hardcoding paths; use parameters for flexibility)
_meta_file_path = dbutils.widgets.get("meta_file_path")
_utilities_folder_path = dbutils.widgets.get("utilities_folder_path")
_log_level = dbutils.widgets.get("log_level").upper()
_task_sys_params_out = dbutils.widgets.get("task_sys_params_out")

# Display the current values of the input parameters for verification and debugging
print(f"Metadata file path : {_meta_file_path}")
print(f"Utilities folder path : {_utilities_folder_path}")
print(f"Log level : {_log_level}")

In [0]:
import sys
import json
sys.path.insert(0, _utilities_folder_path)
from target_writer import *
from audit_logger import *
import traceback
from datetime import datetime, timezone

In [0]:
try:
    meta_parser = TargetWriter(spark=spark)
    source_row_count, affected_rows, updated_rows, deleted_rows, inserted_rows = (
        meta_parser.execute_sql_query(_meta_file_path, _log_level)
    )
    _error_info = {"error_code": "SUCCESS"}
except Exception as ex:
    source_row_count = affected_rows = updated_rows = deleted_rows = inserted_rows = 0
    _error_info = {
        "error_code": "FAILED",
        "error_details": {
            "type": str(type(ex).__name__),
            "filename": str(ex.__traceback__.tb_frame.f_code.co_filename),
            "argument": str(ex.args),
            "location": str(ex.__traceback__.tb_lineno),
            "message": str(ex),
            "debugInfo": str(traceback.format_exc())
        },
    }
    raise
finally:
    if not _task_sys_params_out or _task_sys_params_out.strip() == "":
        _task_sys_params_out = "{}"
    _audit_logger = AuditLogger(
        spark=spark, target_table="sample_superstore.superstore.job_audit_metadata"
    )
    _dml_statistics = {
        "dml_statistics": {
            "source_row_count": f"{source_row_count}",
            "affected_row_count": f"{affected_rows}",
            "updated_row_count": f"{updated_rows}",
            "deleted_row_count": f"{deleted_rows}",
            "inserted_row_count": f"{inserted_rows}",
        },
    }
    _output_stats_data = {
        "dml_statistics": json.dumps(_dml_statistics),
        "error_result": json.dumps(_error_info),
        "_job_end_time_timestamp_ms": f"{int(datetime.now(timezone.utc).timestamp() * 1000)}",
    }
    _audit_logger.audit_write(
        spark=spark,
        sys_metadata_json=_task_sys_params_out,
        stats_data=_output_stats_data,
        log_level=_log_level,
    )